# Naive Bayes Speed Test  OR: Vectorize All the Things!!

It turns out that even fairly simple algorithms like Bernoulli naive Bayes can have dramatic speed differences. Let's demonstrate the difference between a naive (naive naive?!) iterative version in Python (with the help... or possibly hinderance... of Pandas) and a vectorific version that leverages linear algebra.  

So here's the setup.  You've got a spam dataset with roughly 10k words and 5k observations in it, and you want to fit a straightforward Bernoilli naive Bayes to it. This has gotta be the easiest algorithm in all of machine learning, it really is just "estimate the priorities as simple proportions, then slap Bayes rule on it." 

Let's start by loading our (pre-prepared) data and the libraries we'll need.

In [1]:
import pandas as pd
import numpy as np
import toolz
from math import log
import cProfile
df = pd.read_csv("spam_binarized.csv", index_col=0)
df.head()

,LABEL,ibh,national,beside,geting,keen,draw,86021,09061744553,weathers,...,un,runs,chrgd50p,dock,brainy,ages,drugdealer,shore,chuckin,dd
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


So the first column is our label; the rest are our features.  (Note that it's not super aggressively cleaned, I didn't bother to remove numbers from the dictionary, for example---but for a spam classifier, I don't think that's necessarily bad; it's easy to imagine that a number of spam messages contain "800" or other phone number prefixes, for example.)

The first task is to calculate the prior probabilities, which is pretty easy.  We'll just want the proportion of true label values, as well as the conditional probabilities of each feature on both label=1 and label=0.  Since everything is an int in {1, 0}, this is super easy.

In [2]:
prob_y = df["LABEL"].sum() / len(df)
spams = df[df["LABEL"] == 1]
notspams = df[df["LABEL"] == 0]
features = list(df)[1:]

def conditional_probability_dict(column_label, condition_df):
    numerator = condition_df[column_label].sum() + 1 # using laplace smoothing
    denominator = len(condition_df) + 2
    return {column_label: numerator / denominator}

x_probs_conditional_on_spam=[conditional_probability_dict(x, spams) for x in list(spams)[1:]]
x_spam_lookup = toolz.merge(x_probs_conditional_on_spam)

x_probs_conditional_on_notspam=[conditional_probability_dict(x, notspams) for x in list(notspams)[1:]]
x_notspam_lookup = toolz.merge(x_probs_conditional_on_notspam)

Ok, now we have all the basic information we need to use our model. In order to generate a prediction on an observation, we follow: 

$$p(y=\phi | x_{1..n}) = p(y=\phi)\prod_{i=1}^{n}p(x_i | y=\phi)$$ 

After calculating that for both the true and the false cases ($\phi = 1$ and $\phi = 0$), we simply take the max ($argmag_\phi$) and assign our prediction for that document to that case.

(Note that we drop the denominator in the application of Bayes rule because it's the same for both the true and the false cases, so can't affect the comparision.)

Let's start with the naive, slow version. What we're going to do is iterate over all the rows, and, in each row, iterate over the cells.  For each cell, we're going to look up the prior and multiply it out. 

There's one more refinement we have to start with (and which will become important later).  Because the priors are going to get very, very small, in order to avoid ugliness with floating point errors, we should leverage the fact that $log(xy)=log(x)+log(y)$ and instead calculate $log(p(y=\phi)) + \sum_{i=1}^{n}log(p(x_i | y=\phi))$. 

In [3]:
def slow_predict(row):
    prob_spam = log(prob_y)
    prob_notspam = log(1 - prob_y)
    for feat in features:
        if row[feat] == 1:
            prob_spam = prob_spam + log(x_spam_lookup[feat])
            prob_notspam = prob_notspam + log(x_notspam_lookup[feat])
    if prob_spam >= prob_notspam:
        return 1
    else:
        return 0

Now we'll profile this.  Because I know in advance that this will take way too long on the whole dataset, I'll just predict the first, say, 200 rows. 

In [4]:
cProfile.run("df.head(200).apply(slow_predict, axis = 1)")

         63786379 function calls (61853363 primitive calls) in 37.393 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        6    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:996(_handle_fromlist)
      200    1.797    0.009   37.336    0.187 <ipython-input-3-c73fd603c224>:1(slow_predict)
        1    0.000    0.000   37.393   37.393 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 __init__.py:177(iteritems)
        1    0.000    0.000    0.000    0.000 _methods.py:37(_any)
        1    0.000    0.000    0.000    0.000 base.py:1152(_has_complex_internals)
  1932600    2.297    0.000    6.696    0.000 base.py:1303(_convert_scalar_indexer)
        1    0.000    0.000    0.000    0.000 base.py:1351(_convert_slice_indexer)
        1    0.000    0.000    0.000    0.000 base.py:1672(__getitem__)
  1932600    5.715    0.000   31.163    0.000 base.py:2454(get_value)
        3    0.000    0.000  

So that took a while.  38 seconds for 200 rows -> about 15 minutes for 5k rows.  That's totally unreasonable. 

Can we do better?  The key is to observe that once we start adding up logs, the calculation of the posterior on $y=\phi$ for each row is nothing more than the dot product of the log priors for the features and the matrix containing each of those features, plus adding $log(p(y=\phi))$ to each.  (Conveniently, if the word is present for a row, the feature is 1, and the log prior gets included in the dot product; if the word is absent, the dot product multiplies by zero, so there's no need for a conditional to check to see if it's in there.)

For ease of prediction, let's reshape the data a little bit to start.  We'll split out our features from the labels for ease of multiplication, and add a column of all ones at the start to facilitate including $log(p(y=\phi))$ in the dot product rather than adding it later. We'll also change our feature priors from a lookup dict to a Pandas series so that we can multiply it out.

In [5]:
clean_spamdf = df.iloc[:,1:] # get rid of the labels column
clean_spamdf.insert(0, "pseudo_intercept", 1) 
# add column to get posterior on y into dot product.

true_vector =[log(x_spam_lookup[x]) for x in features]
true_vector.insert(0, log(prob_y))
true_vector = pd.Series(true_vector) 
# probably could just keep this as a list, but Pandas series is built on top of 
# numpy anyway, so why not make it clean?

false_vector =[log(x_notspam_lookup[x]) for x in features]
false_vector.insert(0, log(1-prob_y))
false_vector = pd.Series(false_vector)

Ok, as before, we'll predict 200 rows.  For the actual comparison, we *could* just loop over the two dot products, but since we're using numpy for everything, we can just leverage array transformations and comparisons to do it directly.

In [6]:
def fast_predict(df, true_priors, false_priors):
    true_posterior = np.dot(df, true_priors)
    false_posterior = np.dot(df, false_priors)
    combined = np.column_stack((false_posterior, true_posterior)) # basically a transpose
    # reordering so that argmax will return 0 when the first column is max... 
    return np.argmax(combined, axis=1)

cProfile.run("fast_predict(clean_spamdf.head(200), true_vector, false_vector)")

         417 function calls (401 primitive calls) in 0.110 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.092    0.092 <ipython-input-6-7b88fdccb39e>:1(fast_predict)
        1    0.002    0.002    0.110    0.110 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 __init__.py:177(iteritems)
        2    0.000    0.000    0.000    0.000 _methods.py:37(_any)
        1    0.000    0.000    0.000    0.000 base.py:1351(_convert_slice_indexer)
        1    0.000    0.000    0.000    0.000 base.py:1672(__getitem__)
        3    0.000    0.000    0.000    0.000 base.py:3375(_validate_indexer)
        1    0.000    0.000    0.000    0.000 base.py:367(_simple_new)
        2    0.000    0.000    0.000    0.000 base.py:3999(_ensure_index)
        1    0.000    0.000    0.000    0.000 base.py:404(_shallow_copy)
        1    0.000    0.000    0.000    0.000 base.py:522(_reset_identity)
    

0.053 seconds!  That is *7,000 times as fast*!  Let's see how long it takes to cover the whole dataset.

In [7]:
#cProfile.run("fast_predict(clean_spamdf, true_vector, false_vector)")

!!!  Just to make sure we're not cheating, let's actually generate those predictions, and see how many of them match the labels.

In [8]:
#predictions = list(fast_predict(clean_spamdf, true_vector, false_vector))
#true_labels = list(df["LABEL"])
#errors = 0
#matches = 0
#for idx, item in enumerate(true_labels):
#    if predictions[idx] == item:
#        matches += 1
#    else:
#        errors += 1
#print("correct predictions: " + str(matches))
#print("incorrect predictions: " + str(errors))

That's not amazing performance (spams are rare enough that we'd get better accuracy just by predicting everything as not spam), but it's enough to make it plausible that naive Bayes is actually happening here, rather than random senseless math.

So there you are.  The lesson: *vectorize all the things!!*

In [9]:
print(true_vector)

0      -2.009803
1      -5.520127
2      -4.053790
3      -6.618739
4      -6.618739
5      -6.618739
6      -2.955177
7      -4.672829
8      -5.520127
9      -6.618739
10     -6.618739
11     -6.618739
12     -5.925592
13     -6.618739
14     -5.925592
15     -6.618739
16     -6.618739
17     -5.925592
18     -6.618739
19     -6.618739
20     -6.618739
21     -6.618739
22     -6.618739
23     -6.618739
24     -5.925592
25     -5.520127
26     -6.618739
27     -6.618739
28     -5.925592
29     -6.618739
          ...   
9634   -6.618739
9635   -6.618739
9636   -6.618739
9637   -6.618739
9638   -6.618739
9639   -6.618739
9640   -5.925592
9641   -6.618739
9642   -5.925592
9643   -6.618739
9644   -6.618739
9645   -6.618739
9646   -6.618739
9647   -5.925592
9648   -5.925592
9649   -5.925592
9650   -6.618739
9651   -5.520127
9652   -5.925592
9653   -6.618739
9654   -6.618739
9655   -6.618739
9656   -5.925592
9657   -6.618739
9658   -6.618739
9659   -5.925592
9660   -6.618739
9661   -6.6187